## Import Necessary Modules 

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os
from groq import Groq
from langchain_groq import ChatGroq
import ipywidgets as widgets
from IPython.display import display, HTML
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import LLMChain
from langchain_core.documents import Document
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_distances
from langchain_chroma import Chroma
from uuid import uuid4
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.retrievers import BM25Retriever, EnsembleRetriever

### Load Environment Variable

In [6]:
# Load the API key from .env file
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")

## Populating the Vector Database
Check populate_vector_bd.ipynb file

## Load Vector DB or Create Retriever

In [4]:
persist_directory="./Dataset/chroma_db_langchain"
# Embeddings
embedder = HuggingFaceEmbeddings(model_name="BAAI/llm-embedder")

In [5]:
vectordb = Chroma(persist_directory=persist_directory, 
                collection_name="Adavance_RAG_Test",
                embedding_function=embedder)

In [ ]:
# Initialize the ChatGroq model
chatgroq_model = ChatGroq(temperature=0,
                      model_name="deepseek-r1-distill-qwen-32b",
                      api_key=GROQ_API_KEY)

AIMessage(content='<think>\n\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 6, 'total_tokens': 22, 'completion_time': 0.114285714, 'prompt_time': 0.002870991, 'queue_time': 0.051837207999999996, 'total_time': 0.117156705}, 'model_name': 'deepseek-r1-distill-qwen-32b', 'system_fingerprint': 'fp_d458a8aba5', 'finish_reason': 'stop', 'logprobs': None}, id='run-cdad4c26-e3e0-424b-a9e3-d0e4d6e5a8ac-0', usage_metadata={'input_tokens': 6, 'output_tokens': 16, 'total_tokens': 22})

In [ ]:
import os
import requests
import chromadb
from datetime import datetime, timedelta
from langchain.agents import Tool, AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import SystemMessage
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize environment variables
os.environ["SERPER_API_KEY"] = "your_serper_api_key"
OPENMETEO_URL = "https://api.open-meteo.com/v1/forecast"

# ================== Book Analysis Agent ==================
class BookAnalysisAgent:
    def __init__(self):
        # Initialize embedding model
        self.embeddings = HuggingFaceEmbeddings(
            model_name="BAAI/llm-embedder",
            model_kwargs={'device': 'gpu'},
            encode_kwargs={'normalize_embeddings': True}
        )
        
        # Initialize Chroma DB
        self.client = chromadb.PersistentClient(path="book_db")
        self.collection = self.client.get_or_create_collection("books")
        
        # Create LangChain vectorstore
        self.vectorstore = Chroma(persist_directory=persist_directory, 
                collection_name="Adavance_RAG_Test",
                embedding_function=embedder)
        
        self.retriever = self.vectorstore.as_retriever()

    def analyze_book(self, query):
        docs = self.retriever.get_relevant_documents(query)
        return "\n\n".join([f"From {doc.metadata['title']}:\n{doc.page_content}" for doc in docs])

# ================== Internet Search Agent ==================
class InternetSearchAgent:
    def __init__(self):
        self.search = GoogleSerperAPIWrapper()
        self.history = []
        
    def search_web(self, query):
        result = self.search.run(query)
        self.history.append({"query": query, "result": result})
        return result

# ================== Agent Orchestration ==================
# Initialize agents
book_agent = BookAnalysisAgent()
search_agent = InternetSearchAgent()

# Define tools
tools = [
    Tool(
        name="Book Analysis",
        func=book_agent.analyze_book,
        description="Analyzes books 'Verity' and 'The Girl on the Train' using vector database"
    ),
    Tool(
        name="Weather Forecast",
        func=lambda location: weather_agent.get_weather(location),
        description="Provides weather forecasts using Open-Meteo API"
    ),
    Tool(
        name="Web Search",
        func=search_agent.search_web,
        description="Searches the internet using Google via Serper API"
    )
]

# Set up memory and agent
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    "system_message": SystemMessage(content="You are a helpful assistant with access to multiple tools.")
}

memory = ConversationBufferMemory(memory_key="memory", return_messages=True)
llm = ChatOpenAI(temperature=0)  # Replace with your preferred LLM

agent = create_structured_chat_agent(llm, tools, agent_kwargs)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

# ================== Usage Example ==================
if __name__ == "__main__":
    while True:
        try:
            query = input("\nUser: ")
            if query.lower() in ['exit', 'quit']:
                break
                
            result = agent_executor.invoke({"input": query})
            print(f"\nAssistant: {result['output']}")
            
        except KeyboardInterrupt:
            break